In [0]:
# %pip install -U -qqqq mlflow
# %restart_python

In [0]:
from configs.project import get_project_config

In [0]:
projectConfig = get_project_config()
projectConfig.model_dump()

## Create Catalog, Schema, and Volume 

In [0]:
catalog = projectConfig.uc_catalog
schema = projectConfig.uc_schema
data_folder = projectConfig.raw_data_volume

In [0]:
try:
  spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
except Exception as e:
  pass
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")
spark.sql(f"GRANT CREATE, USAGE on DATABASE {catalog}.{schema} TO `account users`")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog}.{schema}.{data_folder}")

## Create a Secret Scope

for admins please see this reference to create a secret-scope: https://docs.databricks.com/aws/en/security/secrets/?language=Databricks%C2%A0workspace%C2%A0UI#create-a-secret-scope

The databricks secret holding at personal access token is used for deploying the model with agent that accesses the Genie space.  Users can use the code below to store their PAT in a databricks managed secret.


In [0]:
create_scope = False
if create_scope:
    secret_scope = next(
        (
            item
            for item in dbutils.secrets.listScopes()
            if projectConfig.secret_scope == item.name
        ),
        None,
    )

    print(secret_scope)

In [0]:
cleanup = False
if cleanup:
    dbutils.secrets.deleteScope(secret_scope)

## Create a Secret Value

In [0]:
create_secret = False
if create_secret:
    import os
    from getpass import getpass

    print("enter your secret key")
    os.environ["my_secret"] = getpass()

In [0]:
if create_secret:
    from databricks.sdk import WorkspaceClient

    w = WorkspaceClient(
    #   host="https://e2-demo-field-eng.cloud.databricks.com/",
    #   client_id=dbutils.secrets.get("felix-flory","SERVICE_PRINCIPAL_ID"),
    #   client_secret=dbutils.secrets.get("felix-flory","SERVICE_PRINCIPAL_SECRET"),
    )

    w.secrets.put_secret(
        projectConfig.secret_scope, "<key-name>", string_value=os.environ["my_secret"]
    )

## Create External Serving Endpoints

This step is optional and creates an external LLM serving endpoint. Two examples are commented out: one of OpenAI and one for Anthropic hosted LLM endpoints.

In [0]:
import mlflow.deployments
from requests.exceptions import HTTPError

client = mlflow.deployments.get_deploy_client("databricks")


In [0]:
# endpoint_name = projectConfig.llm_endpoint_names[0]
# endpoint_list = [
#     e
#     for e in client.list_endpoints()
#     if "fflory" in e.get("name", "") or endpoint_name in e.get("name", "")
# ]

In [0]:
# endpoint_name = projectConfig.llm_endpoint_names[0]
# ep_config = {
#     "name": endpoint_name,
#     "config": {
#         "served_entities": [
#             {
#                 "external_model": {
#                     "name": "gpt-4o",
#                     "provider": "openai",
#                     "task": "llm/v1/chat",
#                     "openai_config": {
#                         "openai_api_key": f"{{{{secrets/{projectConfig.secret_scope}/OPENAI_API_KEY}}}}"
#                     },
#                 },
#             },
#         ],
#         "route_optimized": True,
#     },
# }

# try:
#     client.get_endpoint(endpoint_name)
#     print(f"Endpoint {endpoint_name} already exists")
# except HTTPError as e:
#     print("Endpoint does not exist. Creating it.")
#     client.create_endpoint(
#         # name=endpoint_name,
#         config=ep_config,
#     )

In [0]:
# endpoint_name = projectConfig.llm_endpoint_names[1]
# ep_config = {
#     "name": endpoint_name,
#     "config": {
#         "served_entities": [
#             {
#                 "external_model": {
#                     "name": "claude-3-7-sonnet-20250219",
#                     "provider": "anthropic",
#                     "task": "llm/v1/chat",
#                     "anthropic_config": {
#                         "anthropic_api_key": f"{{{{secrets/{projectConfig.secret_scope}/ANTHROPIC_API_KEY}}}}"
#                     },
#                 },
#             },
#         ],
#         "route_optimized": True,
#     },
# }

# try:
#     client.get_endpoint(endpoint_name)
#     print(f"Endpoint {endpoint_name} already exists")
# except HTTPError as e:
#     print("Endpoint does not exist. Creating it.")
#     client.create_endpoint(
#         # name=endpoint_name,
#         config=ep_config,
#     )

## Create MLFlow Experiment Workspace path

In [0]:
from src.utils import set_mlflow_experiment

experiment = set_mlflow_experiment(projectConfig.mlflow_experiment_name)

In [0]:
experiment.to_proto()